In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [35]:
# Loading the dataset to a dataframe
data = pd.read_excel('950woNames.xlsx')

# converting the String columns into int columns.
# this is done beacause the model can read only numerical values
le = LabelEncoder()
data['File Name'] = le.fit_transform(data['File Name'])

# Assigning the data into variables
X = data[['carbs', 'calories', 'fiber', 'fat', 'protein', 'File Name', 'CGM_Data']]
print(X)
Y = data['After_BGL']

# Saving the dataframe to the file for future reference
data.to_excel('950woNames.xlsx', index=False)

     carbs  calories  fiber    fat  protein  File Name  CGM_Data
0    14.83     79.80   0.81   1.00     2.66          0     118.8
1    31.90    520.68   7.73  28.68    37.52          0     154.8
2    77.15    581.48   7.74  19.97    24.31          0     154.8
3    46.56    456.75   7.36  20.04    25.46          0     109.8
4    58.57    496.23   6.01  13.53    32.29          0     156.6
..     ...       ...    ...    ...      ...        ...       ...
944  20.92    123.30   2.21   1.65     6.60         40     144.0
945  68.32    587.59  15.25  19.27    36.81         40     165.6
946  98.60    655.59   6.72  15.23    29.19         40     124.2
947  15.69     65.27   3.21   0.56     1.22         40     122.4
948  93.47    714.16  23.72  19.18    43.35         40     135.0

[949 rows x 7 columns]


In [30]:
# Splitting the dataset into train and test data

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
# Normalizing the input columns
normalizer = layers.Normalization()
normalizer.adapt(X_train)

# Building a linear regression model using keras
model = keras.Sequential([
  normalizer,
  layers.Dense(1)
])

# compiling the model with Adam optimiser
model.compile(loss='mean_squared_error', optimizer=tf.optimizers.Adam(0.1))

# Training the model for 100 epochs
history = model.fit(X_train, y_train, epochs=100,validation_data=(X_test, y_test))

Epoch 1/100
24/24 [==============================] - 0s 5ms/step - loss: 32535.0918 - val_loss: 32042.9141
Epoch 2/100
24/24 [==============================] - 0s 2ms/step - loss: 31611.0254 - val_loss: 31115.7754
Epoch 3/100
24/24 [==============================] - 0s 2ms/step - loss: 30708.5801 - val_loss: 30233.1660
Epoch 4/100
24/24 [==============================] - 0s 2ms/step - loss: 29850.0508 - val_loss: 29364.1152
Epoch 5/100
24/24 [==============================] - 0s 2ms/step - loss: 28993.5156 - val_loss: 28544.2930
Epoch 6/100
24/24 [==============================] - 0s 2ms/step - loss: 28189.3438 - val_loss: 27728.9043
Epoch 7/100
24/24 [==============================] - 0s 2ms/step - loss: 27393.7461 - val_loss: 26929.6289
Epoch 8/100
24/24 [==============================] - 0s 2ms/step - loss: 26616.1113 - val_loss: 26161.5508
Epoch 9/100
24/24 [==============================] - 0s 2ms/step - loss: 25864.7930 - val_loss: 25421.5098
Epoch 10/100
24/24 [=================

In [37]:
# Evaluating the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print('loss:', test_loss)
print('MSE:', mse)
print('R^2:', r2)

6/6 [==============================] - 0s 719us/step
loss: 1932.8216552734375
MSE: 1932.8217137747506
R^2: 0.22914587133328046


In [38]:
# testing the model manually with a sample input
x_input = [[24.83, 9.80, 0.81, 1.00, 2.66, 77,118.8]]
y_pred_test = model.predict(x_input)
print(y_pred_test)

1/1 [==============================] - 0s 26ms/step
[[150.61667]]


In [39]:
# converting the model into a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('linear_regression_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\THUSHA~1\AppData\Local\Temp\tmpo8_grlf7\assets


INFO:tensorflow:Assets written to: C:\Users\THUSHA~1\AppData\Local\Temp\tmpo8_grlf7\assets
